In [5]:
from torch_geometric.datasets import Planetoid
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

#A two-layer GCN classification example with CORA dataset

dataset = Planetoid(root='/tmp/Cora', name='Cora')
print('Dataset info:')
print(dataset)
num_node_features = dataset.num_node_features #The number of node features (the input of GCN)
dim_hidden = 16 #The hidden layer's dimension of GCN
dim_output = dataset.num_classes #The output's dimension
num_epoch = 200 #The number of epoch
learning_rate = 0.01 #The learning rate for GCN training
print('-----------------------')
print('The number of node features:{}\nThe dimension of hidden layer{}\nThe dimension of output layer:{}'.format(num_node_features, dim_hidden, dim_output))
print('-----------------------')

Dataset info:
Cora()
-----------------------
The number of node features:1433
The dimension of hidden layer16
The dimension of output layer:7
-----------------------


In [8]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(num_node_features, dim_hidden) #The first layer of GCN
        self.conv2 = GCNConv(dim_hidden, dim_output) #The second layer of GCN
        
    def forward(self, data):
        #data.x: the node features
        #data.edge_index: the edges of graph
        x, edge_index = data.x, data.edge_index 
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)
        
device = torch.device('cpu') 
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=5e-4)

loss_log = []

In [7]:

#training part
model.train()
for epoch in range(num_epoch):
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output[data.train_mask], data.y[data.train_mask])
    loss_log.append(loss.item())
    loss.backward()
    optimizer.step()

#test part
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
accuracy = int(correct) / int(data.test_mask.sum())
print('Number of test sample:{}\nNumber of correct prediction:{}\nThe accuracy:{:.4f}'.format(data.test_mask.sum(),correct,accuracy))



Number of test sample:1000
Number of correct prediction:802
The accuracy:0.8020
